In [27]:
from util.utils import *
import os

In [28]:
## load datasets
def loadDataframes():
    cat62 = None
    #cat62 = openDataframe("cat62_bronze.csv")
    espera = openDataframe("espera_bronze.csv")
    bimtra = openDataframe("bimtra_bronze.csv")
    metaf = openDataframe("metaf_bronze.csv")
    metar = openDataframe("metar_bronze.csv")
    tc_prev = openDataframe("tc-prev_bronze.csv")
    tc_real = openDataframe("tc-real_bronze.csv")
    return cat62, espera, bimtra, metaf, metar, tc_prev, tc_real

cat62, espera, bimtra, metaf, metar, tc_prev, tc_real = loadDataframes()

In [29]:
tc_real['aero'] = tc_real['aero'].apply(lambda x: 'SB' + x)

In [30]:
# MERGE BIMTRA - TC_REA
resultado = bimtra.merge(tc_real, left_on='origem', right_on='aero', how='inner')
print(resultado.shape)

# Filtrar usando uma função lambda após a mesclagem
resultado = resultado.loc[abs(resultado['dt_dep'] - resultado['hora']) <= 1800000]

# Renomear as colunas
resultado = resultado.rename(columns={'hora': 'hora_real', 'aero': 'aero_real', 'hora_formatted': 'hora_formatted_real'})


(54819048, 12)


In [31]:
resultado.shape

(33491, 12)

In [32]:
# MERGE COM RESULTADO E TC PREAV AGORA
tc_prev['aero'] = tc_prev['aero'].apply(lambda x: 'SB' + x)

In [33]:
# MERGE BIMTRA - TC_PREV
resultado2 = resultado.merge(tc_prev, left_on='destino', right_on='aero', how='inner')

# Filtrar usando uma função lambda após a mesclagem
resultado2 = resultado2.loc[abs(resultado2['hora'] - resultado2['dt_arr']) <= 1800000]

# Renomear as colunas
resultado2 = resultado2.rename(columns={'hora': 'hora_prev', 'hora_formatted': 'hora_formatted_prev', 'aero': 'aero_prev'})


In [34]:
# MERGE RESULTADO2 - METAR
resultado3 = resultado2.merge(metar, left_on='origem', right_on='aero', how='inner')

# Filtrar usando uma função lambda após a mesclagem
resultado3 = resultado3.loc[abs(resultado3['dt_dep'] - resultado3['hora']) <= 1800000]

# Renomear as colunas
resultado3 = resultado3.rename(columns={'hora': 'hora_metar', 'hora_formatted': 'hora_formatted_metar', 'aero': 'aero_metar'})


In [35]:
resultado3.shape

(45247, 20)

In [36]:
# MERGE RESULTADO3 - METAF
resultado4 = resultado3.merge(metaf, left_on='destino', right_on='aero', how='inner')

# Filtrar usando uma função lambda após a mesclagem
resultado4 = resultado4.loc[abs(resultado4['hora'] - resultado4['dt_arr']) <= 1800000]

# Renomear as colunas
resultado4 = resultado4.rename(columns={'hora': 'hora_metaf', 'hora_formatted': 'hora_formatted_metaf', 'aero': 'aero_metaf'})


In [37]:
resultado4.shape

(38113, 24)

In [38]:
resultado4.head()

,flightid,origem,destino,dt_dep,dt_arr,dt_dep_formatted,dt_arr_formatted,hora_real,nova_cabeceira,antiga_cabeceira,...,aero_prev,hora_formatted_prev,hora_metar,metar,aero_metar,hora_formatted_metar,hora_metaf,metaf,aero_metaf,hora_formatted_metaf
1,ffb9ee53437196c08c6329ec5a50a3e1,SBCT,SBKP,1675211054000,1675214748000,"Tuesday, January 31, 2023 21:24:14","Tuesday, January 31, 2023 22:25:48",1675209845000,33,15,...,SBKP,"Tuesday, January 31, 2023 22:00:00",1675209600000,METAR SBCT 010000Z 02003KT 9999 FEW030 SCT100 ...,SBCT,"Tuesday, January 31, 2023 21:00:00",1675213200000,METAF SBKP 010100Z 10007KT 6000 - RA BKN033...,SBKP,"Tuesday, January 31, 2023 22:00:00"
2046,27aef8f18a47f429df6f9a9c91cef8c5,SBCT,SBKP,1675292483000,1675294999000,"Wednesday, February 01, 2023 20:01:23","Wednesday, February 01, 2023 20:43:19",1675292144000,15,33,...,SBKP,"Wednesday, February 01, 2023 21:00:00",1675292400000,METAR SBCT 012300Z 09005KT 9999 SCT045 22/22 Q...,SBCT,"Wednesday, February 01, 2023 20:00:00",1675296000000,METAF SBKP 020000Z 09005KT 6000 BKN025 ...,SBKP,"Wednesday, February 01, 2023 21:00:00"
4078,101feb32f21cb8722639c3b8cd54acc0,SBCT,SBKP,1675329948000,1675332840000,"Thursday, February 02, 2023 06:25:48","Thursday, February 02, 2023 07:14:00",1675329953000,33,15,...,SBKP,"Thursday, February 02, 2023 07:00:00",1675328400000,METAR SBCT 020900Z 32005KT 9999 BKN010 20/20 Q...,SBCT,"Thursday, February 02, 2023 06:00:00",1675332000000,METAF SBKP 021000Z 35005KT 2000 RABR OVC02...,SBKP,"Thursday, February 02, 2023 07:00:00"
6106,f24ce5831353c8e053986b4dd2a39bf7,SBCT,SBKP,1675350095000,1675352830000,"Thursday, February 02, 2023 12:01:35","Thursday, February 02, 2023 12:47:10",1675349277000,29,33,...,SBKP,"Thursday, February 02, 2023 13:00:00",1675350000000,METAR SBCT 021500Z 34011KT 9999 OVC025 25/24 Q...,SBCT,"Thursday, February 02, 2023 12:00:00",1675353600000,METAF SBKP 021600Z 29009KT 9999 BKN033 ...,SBKP,"Thursday, February 02, 2023 13:00:00"
8128,f24ce5831353c8e053986b4dd2a39bf7,SBCT,SBKP,1675350095000,1675352830000,"Thursday, February 02, 2023 12:01:35","Thursday, February 02, 2023 12:47:10",1675349697000,33,29,...,SBKP,"Thursday, February 02, 2023 13:00:00",1675350000000,METAR SBCT 021500Z 34011KT 9999 OVC025 25/24 Q...,SBCT,"Thursday, February 02, 2023 12:00:00",1675353600000,METAF SBKP 021600Z 29009KT 9999 BKN033 ...,SBKP,"Thursday, February 02, 2023 13:00:00"


In [39]:
resultado4.to_csv("../data/processed_data/merged_bronze_02.csv")